In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as sps

# Project, exploring the Hubble "Tension"

Recall that in weeks 1 and 2 of this course we explored some measurements of the Hubble constant and used them to study some simple statistics, and to uise inverse variance weighting to make a weighted average of measurements that took into account the fact that the some of the measurements had smaller uncertainties than others.

It turns out that the consistency (or potential inconsistency) is the measurements of the Hubble constant is actually one of the biggest open questions in cosmology.  Roughly speaking, measurements of the Hubble constant can be divided into two types. 

1. "Early time" measurements that use our knowledge of cosmology, and look at structures in the very early universe, e.g., using the primoridal background of light left over from the Big Bang that is now visible to us as a bath of faint microwave radiation, to determine the Hubble parameter.  These measurements are more sensitive to our understanding of cosmology.

2. "Late time" measurements that use supernovae and build on the so-called distance ladder to estimate the distance to the supernovae, and then compare that to the redshift of their spectral lines (i.e., the sort of measurement we did in week 6 looking at the spectrum of a Galaxy using SDSS data).  These measurements are more sensitive to our ability to build up a distance ladder out to billions of light years away.

We've provided you with a new data file, which is a sub-set of the more recent Hubble data measurements, including an extra column telling you if a particular measurement is type 1 or type 2.

Here are some things that you might try and do:

1. Make a nice plot that summarize the difference between "Early Time" results and "Late Time" results

2. Come up with some way to estimate how consistent the data within each subset ("Early Time" and "Late Time") are with themselves.

3. Come up with some way to estimate how consistent the two different subsets ("Early Time" and "Late Time") are with each other.



In [ ]:
data = np.loadtxt(open("../data/Hubble_Extra.txt", 'rb'), usecols=[1,2,3,4])

In [ ]:
# This is how we pull out the data from columns in the array.
H0_measured = data[:,0]
H0_errorLow = data[:,1]
H0_errorHigh = data[:,2]
H0_type = data[:,3]
N_measurements = H0_measured.size

# Here a couple of useful functions.

The first one pulls out some useful statistical quantities, given arrays of values and associated error estimates

The second one takes a set of Hubble parameter measurements, and the associated error bars (both low and high side) and prints out the statistical quantities and also make a plot of the measurments.

In [ ]:
def getStats(values, errors):
    nvals = np.size(values)
    mean = np.mean(values)
    std = np.std(values)
    err = std / np.sqrt(nvals)
    wts = 1/(errors*errors)
    wmean = np.sum(values*wts)/np.sum(wts)
    werr = np.sqrt(1/np.sum(wts))
    return (nvals, mean, std, err, wmean, werr)

In [ ]:
def getStatsAndPlotValues(values, errs_lo, errs_hi):
    errs = 0.5*(errs_lo + errs_hi)
    stats = getStats(values, errs)
    print("Statistics for these data ----")
    print("N                      : %i" % stats[0])
    print("Mean                   : %0.2f" % stats[1])
    print("Standard Deviation     : %0.2f" % stats[2])
    print("Standard Error         : %0.2f" % stats[3])
    print("Weighted Mean          : %0.2f" % stats[4])
    print("Error on Weighted Mean : %0.2f" % stats[5])
    _ = plt.errorbar(values, np.arange(stats[0]), xerr=(errs_lo, errs_hi), fmt=".", color='k')
    _ = plt.xlabel("Hubble Constant [km/s/Mpc]")
    _ = plt.ylabel("Experiment number")
    _ = plt.errorbar(stats[1], stats[0]/2., xerr=stats[3], yerr=stats[0]/2, fmt='o', color='b', label="Mean")
    _ = plt.errorbar(stats[4], stats[0]/2., xerr=stats[5], yerr=stats[0]/2, fmt='o', color="r", label="Weighted Mean")
    _ = plt.legend()

In [ ]:
getStatsAndPlotValues(H0_measured, H0_errorLow, H0_errorHigh)

# Masking data by measurement type

This next cell shows how to select measurements of either type

In [ ]:
type_1_mask = H0_type == 1
type_2_mask = H0_type == 2
print("Early time measurements", H0_measured[type_1_mask])
print("Late time measurements", H0_measured[type_2_mask])

# Going from significance in sigma to p-value and vice versa

These functions might be useful.

In [ ]:
sigma_value = 3
p_value = sps.norm().sf(sigma_value)
sigma_check = sps.norm().isf(p_value)
print("Original sigma : %.2f" % sigma_value)
print("P-value        : %.2e" % p_value)
print("Back to sigma  : %.2f" % sigma_check)

# Going from a chi-squared for n degrees for freedom to a p-value

This function tells you the p-value to observe total chi squared value given df = n-1 measurements.

Where df is the "degrees of freedom", in our case, we might have, say, 11 measuements, but then we are picking one value of the Hubble constant to try and match all the data, so that leaves us with 10 "degrees of freedom".

I.e., if we had a 11 measurements, and the total chi squared summed up to 11.5, then the p-value would be about 0.36/

In [ ]:
print("The p-value for chi squared = 11.5 with 10 degrees of freedom is %0.2f" % sps.chi2(df=10).sf(11.5))